<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)

#  Introduction to using [Apache Spark](https://spark.apache.org/) with the [PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)

### Attributes of a `sparkContext`

You can use Python's [dir()](https://docs.python.org/2/library/functions.html?highlight=dir#dir) function to get a list of all the attributes (including methods) accessible through the `sqlContext` object.

In [ ]:
# List sqlContext's attributes
dir(sqlContext)

### Getting help on functions and attributes

1. Python's [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) function is used to get an easier to read list of all the attributes, including examples, that the `sqlContext` object has.
2. Jupyter notebook's `?` feature also does the same

In [ ]:
# Use help to obtain more detailed information
help(sqlContext)

In [ ]:
sqlContext?

# Using DataFrames and chaining together transformations and actions

## Working with DataFrames

In Spark, we first create a base [DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame). We can then apply one or more **transformations** to that base DataFrame. *A DataFrame is immutable, so once it is created, it cannot be changed.* As a result, each transformation creates a new DataFrame. Finally, we can apply one or more **actions** to the DataFrames.

### Contents of a DataFrame

A DataFrame consists of a series of `Row` objects; each `Row` object has a set of named columns. You can think of a DataFrame as modeling a table, though the data source being processed does not have to be a table.

Therefore, a DataFrame must have a _schema_, which means it must consist of columns, each of which has a _name_ and a _type_. Some data sources have schemas built into them. Examples include RDBMS databases, Parquet files, and NoSQL databases like Cassandra. Other data sources don't have computer-readable schemas, but you can often apply a schema programmatically.

### *Example*: Create a Python collection of 10,000 people

We will use a third-party Python testing library called [fake-factory](https://pypi.python.org/pypi/fake-factory/0.5.3) to create a collection of fake person records.

In [ ]:
from faker import Factory
fake = Factory.create()
fake.seed(4321)

In [ ]:
# Each entry consists of last_name, first_name, SSN, job, and age (at least 1)
from pyspark.sql import Row
def fake_entry():
  name = fake.name().split()
  return (name[1], name[0], fake.ssn(), fake.job(), abs(2016 - fake.date_time().year) + 1)

In [ ]:
# Create a helper function to call a function repeatedly
def repeat(times, func, *args, **kwargs):
    for _ in range(times):
        yield func(*args, **kwargs)

In [ ]:
data = list(repeat(10000, fake_entry))

`data` is just a normal Python list, containing Python tuples objects. Let's look at the first item in the list:

In [ ]:
data[0]

We can check the size of the list using the Python `len()` function.

In [ ]:
len(data)

### Distributing a Spark SQL DataFrame among slaves

In Spark, datasets are represented as a list of entries, where the list is broken up into many different partitions that are each stored on a different machine.  Each partition holds a unique subset of the entries in the list.  Spark calls datasets that it stores "Resilient Distributed Datasets" (RDDs). Even DataFrames are ultimately represented as RDDs, with additional meta-data.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3b.png" style="width: 900px; margin: 5px"/>

In [ ]:
sqlContext.createDataFrame?

In [ ]:
dataDF = sqlContext.createDataFrame(data, ('last_name', 'first_name', 'ssn', 'occupation', 'age'))

### What is dataDF?

In [ ]:
print(type(dataDF))

### Checking the schema of a dataframe

In [ ]:
dataDF.printSchema()

### Registering a DataFrame as an SQL Table

In [ ]:
sqlContext.registerDataFrameAsTable(dataDF, 'dataframe')

What methods can we call on this DataFrame?

In [ ]:
help(dataDF)

How many partitions will the DataFrame be split into?

In [ ]:
dataDF.rdd.getNumPartitions()

### Using the _select_ transformation

One of the most common DataFrame operations is `select()`, and it works more or less like a SQL `SELECT` statement: You can select specific columns from the DataFrame, and you can even use `select()` to create _new_ columns with values that are derived from existing column values. We can use `select()` to create a new column that decrements the value of the existing `age` column.

`select()` is a _transformation_. It returns a new DataFrame that captures both the previous DataFrame and the operation to add to the query (`select`, in this case). But it does *not* actually execute anything on the cluster.

In [ ]:
# Transform dataDF through a select transformation and rename the newly created '(age -1)' column to 'age'
# Because select is a transformation and Spark uses lazy evaluation, no jobs, stages,
# or tasks will be launched when we run this code.
subDF = dataDF.select('last_name', 'first_name', 'ssn', 'occupation', (dataDF.age - 1).alias('age'))

### Use _collect_ to view results (the _reduce_ operation)

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3d.png" style="height:700px"/>

The `collect()` method is an **action**

In [ ]:
# Let's collect the data
results = subDF.collect()

Visualizing the data

In [ ]:
subDF.show()  # by default, 20 rows

Displaying full column width

In [ ]:
subDF.show(n=10, truncate=False)

### Using the _count_ action

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3e.png" style="height:700px;"/>

In [ ]:
print(dataDF.count())
print(subDF.count())

### Using the _filter_ transformation, Similar to the WHERE clause in SQL

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3f.png" style="height:700px;"/>

In [ ]:
filteredDF = subDF.filter(subDF.age < 10)
filteredDF.show(truncate=False)
filteredDF.count()

## Exercise: Display individuals with last name 'Brown' and older than 20, count their number
### Hint: Use the `filter` and `count` operations

In [ ]:
# enter code here

## Arbitrary User Defined Functions
### (because SQL cannot do everything)

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
from pyspark.sql.types import BooleanType
less_ten = udf(lambda s: s < 10, BooleanType())
lambdaDF = subDF.filter(less_ten(subDF.age))
lambdaDF.show()
lambdaDF.count()

In [ ]:
# Collect the even values less than 10
even = udf(lambda s: s % 2 == 0, BooleanType())
evenDF = lambdaDF.filter(even(lambdaDF.age))
evenDF.show()
evenDF.count()

## Exercise: Use the following Python function as a UDF to filter the dataframe
### What does the function do? On which column should it be applied?

In [ ]:
def exercise_udf(s):
    return 'engineer' in s.lower()

In [ ]:
# enter code here

## Additional DataFrame actions

Some additional actions:

* [first()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.first)
* [take()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.take)

One useful thing to do when we have a new dataset is to look at the first few entries to obtain a rough idea of what information is available.  In Spark, we can do that using actions like `first()`, `take()`, and `show()`. Note that for the `first()` and `take()` actions, the elements that are returned depend on how the DataFrame is *partitioned*.

Instead of using the `collect()` action, we can use the `take(n)` action to return the first _n_ elements of the DataFrame. The `first()` action returns the first element of a DataFrame, and is equivalent to `take(1)[0]`.

In [ ]:
from pprint import pprint
pprint(filteredDF.first())
print('=' * 80)
pprint(filteredDF.take(4))

This looks better:

In [ ]:
filteredDF.show()

## Additional DataFrame transformations

### _orderBy_

[`orderBy()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) allows you to sort a DataFrame by one or more columns, producing a new DataFrame.

This returns a `Column`, which has additional methods like `desc()` (for sorting in descending order) or `asc()` (for sorting in ascending order, which is the default).

In [ ]:
# Get the five oldest people in the list. To do that, sort by age in descending order.
pprint(dataDF.orderBy(dataDF.age.desc()).take(5))

#### For ascending order (default)

In [ ]:
pprint(dataDF.orderBy('age').take(5))

### _distinct_ and _dropDuplicates_

[`distinct()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) filters out duplicate rows, and it considers all columns. Since our data is completely randomly generated (by `fake-factory`), it's extremely unlikely that there are any duplicate rows:

### Q: Are there any duplicates in the dataset?

In [ ]:
print(dataDF.count())
print(dataDF.distinct().count())

To demonstrate `distinct()`, let's create a quick throwaway dataset.

In [ ]:
tempDF = sqlContext.createDataFrame([("Joe", 1), ("Joe", 1), ("Anna", 15), ("Anna", 12), ("Ravi", 5)],
                                    ('name', 'score'))

In [ ]:
tempDF.show()

In [ ]:
tempDF.distinct().show()

Note that one of the ("Joe", 1) rows was deleted, but both rows with name "Anna" were kept, because all columns in a row must match another row for it to be considered a duplicate.

[`dropDuplicates()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates) is like `distinct()`, except that it allows us to specify the columns to compare.

In [ ]:
print(dataDF.count())
print(dataDF.dropDuplicates(['first_name', 'last_name']).count())

## Exercise: How many different values of age are there?

In [ ]:
# enter code here

### _drop_

[`drop()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop) is like the opposite of `select()`: Instead of selecting specific columns from a DataFrame, it drops a specifed column from a DataFrame.

In [ ]:
dataDF.drop('occupation').drop('age').show()

### _groupBy_

[`groupBy()`]((http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) is one of the most powerful transformations. It allows you to perform aggregations on a DataFrame.

Unlike other DataFrame transformations, `groupBy()` does _not_ return a DataFrame. Instead, it returns a special [GroupedData](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) object that contains various aggregation functions.

The most commonly used aggregation function is [count()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count),
but there are others (like [sum()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum), [max()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.max), and [avg()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.avg).

In [ ]:
dataDF.groupBy('occupation').count().show(truncate=False)

In [ ]:
dataDF.groupBy().avg('age').show(truncate=False)

We can also use `groupBy()` to do aother useful aggregations:

In [ ]:
print("Maximum age: {0}".format(dataDF.groupBy().max('age').first()[0]))
print("Minimum age: {0}".format(dataDF.groupBy().min('age').first()[0]))

## Exercise: Sort occupations by average age

In [ ]:
# enter code here